In [ ]:
#pip install webdriver_manager

In [ ]:
#pip install splinter

In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
#from splinter.exceptions import ElementDoesNotExist
import requests
import pandas as pd
import re
import time

# Mac Users

In [ ]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

In [ ]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

# Windows Users

In [ ]:
# executable_path = {'executable_path': 'chromedriver.exe'}
# browser = Browser('chrome', **executable_path, headless=False)

In [2]:
#We are installing the webdriver (robot)
executable_path = {"executable_path": ChromeDriverManager().install()}
#Splinter is creating a browser window with the robot
browser = Browser("chrome", **executable_path, headless=False)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\223005380\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


#### NASA Mars News

In [ ]:
browser.visit("https://mars.nasa.gov/news/")

In [ ]:
html = browser.html
#html

In [ ]:
# Create a Beautiful Soup object
soup_Mars_News = bs(html, 'html.parser')
type(soup_Mars_News)

In [ ]:
#print(soup_Mars_News.prettify())

In [ ]:
news_title = []
news_p = []
mars = {}

#iterating to get the title from the beautiful soup object
news = soup_Mars_News.find_all("div", class_="bottom_gradient")
for n in news:
    #print(n.text)
    news_title.append(n.text)
#print(len(news_title))

#iterating to get the paragraph from the beautiful soup object
news = soup_Mars_News.find_all("div", class_="article_teaser_body")
for n in news:
    #print(n.text)
    news_p.append(n.text)
#print(len(news_p))

#The following creates a Data Frame (Commented because it is not required)
# news_df = {"Title":news_title,"Paragraph":news_p} 
# news_df = pd.DataFrame(news_l, columns=['Title','Paragraph'])
# news_df

#Just display the first item
mars["news_title"]= news_title[0]
mars["news_p"]= news_p[0]
mars

#### JPL Mars Space Images - Featured Image

In [ ]:
browser.visit("https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars")

In [ ]:
html = browser.html
#html

In [ ]:
# Create a Beautiful Soup object
soup_Mars_Images = bs(html, 'html.parser')
type(soup_Mars_Images)

In [ ]:
#print(soup_Mars_Images.prettify())

In [ ]:
#Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url
#string to a variable called `featured_image_url`.

#NOTE: Through this code I extracted the medium size image (not required), next cell contains the largesize image 
#The image is contained in a div class "carousel_items" in a <a> class "buttom fancybox"
featured_image_url2 = soup_Mars_Images.find("a", class_="button fancybox").get('data-fancybox-href')
print(featured_image_url2)
featured_image_url2 = f'https://www.jpl.nasa.gov{featured_image_url2}'
featured_image_url2

In [ ]:
#Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url
#string to a variable called `featured_image_url`.

#NOTE: This code extracts the LargeImage and save it in to a key of the Mars dictionary
browser.visit("https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars")
time.sleep(2)
browser.find_by_id("full_image").click()
time.sleep(2)
browser.links.find_by_partial_text("more info").click() #find_link_by_partial_text
time.sleep(2)
soup_mars_images = bs(browser.html, 'html.parser')
result = soup_mars_images.find("figure", class_="lede").a.img["src"]
result =f'https://www.jpl.nasa.gov{result}'
mars["featured_image"]=result
print(result)

### Mars Weather

In [ ]:
browser.visit("https://twitter.com/marswxreport?lang=en")
time.sleep(1)
# browser.find_by_css('.css-901oao').links.find_by_partial_text('InSight')

browser.execute_script("window.scrollTo(1, document.body.scrollHeight);")

#browser.links.find_by_partial_text('InSight')
#time.sleep(2)
#browser.reload()

In [ ]:
#html = browser.html

In [ ]:
# Create a Beautiful Soup object
soup_tweet = bs(browser.html, 'html.parser')
type(soup_tweet)

In [ ]:
print(soup_tweet.prettify())

In [ ]:
#scrape the latest Mars weather tweet. Save the tweet text for the weather report as a variable called `mars_weather`
mars_weather = soup_tweet.find("span", text=re.compile('InSight '))# class_=re.compile('css-901oao'), .span[text=re.compile('InSight')] #, text=re.compile('InSight'))
mars["weather"]=mars_weather.string
print(mars_weather.string)
mars["weather"]

In [ ]:
mars_weather2 = soup_tweet.find_all('span', class_=re.compile('css-901oao'), text=re.compile('InSight'))
for x in mars_weather2:
    print(x.string)
# mars_weather = soup_tweet.find("span", text=re.compile('InSight'))
#print(mars_weather)

### Mars Facts

In [ ]:
url = 'https://space-facts.com/mars/'

In [ ]:
#use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
tables = pd.read_html(url)
tables

In [ ]:
table1 = tables[0]
table1.columns = ['Fact', 'Value']
table1.set_index('Fact', inplace=True)
table1

In [ ]:
table2 = tables[1]
table2.rename(columns={'Mars - Earth Comparison': 'Facts'}, inplace=True)
table2.set_index('Facts', inplace=True)
table2

In [ ]:
#Use Pandas to convert the data to a HTML table string.
html_table1 = table1.to_html()
print(html_table1)
mars["facts"]=html_table1

In [ ]:
table1.to_html('Resources/Raw_table1.html')

In [ ]:
#Use Pandas to convert the data to a HTML table string.
html_table2 = table1.to_html()
print(html_table2)

In [ ]:
table2.to_html('Resources/Raw_table2.html')

### Mars Hemispheres

In [ ]:
#click each of the links to the hemispheres in order to find the image url to the full resolution image

In [ ]:
#Save both the image url string for the hemisphere image, and the Hemisphere title containing the hemisphere name
#Use a Python dictionary to store the data using the keys `img_url` and `title`.

In [ ]:
#Append the dictionary with the image url string and the hemisphere title to a list. 
#This list will contain one dictionary for each hemisphere.
# hemisphere_image_urls = [
#     {"title": "Valles Marineris Hemisphere", "img_url": "..."},
#     {"title": "Cerberus Hemisphere", "img_url": "..."},
#     {"title": "Schiaparelli Hemisphere", "img_url": "..."},
#     {"title": "Syrtis Major Hemisphere", "img_url": "..."},
# ]

In [3]:
browser.visit("https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars")

In [4]:
html = browser.html
#html

In [5]:
# Create a Beautiful Soup object
soup_hemispheres = bs(html, 'html.parser')
type(soup_hemispheres)

bs4.BeautifulSoup

In [ ]:
#print(soup_hemispheres.prettify())

In [32]:
hem_title = soup_hemispheres.find_all('h3')
for x in range(0, len(hem_title)):
    hem_title[x]= hem_title[x].text
hem_title    

['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [ ]:
#Low resolution images at home page
lr_img = soup_hemispheres.find_all('img', class_="thumb")
for link in lr_img:
    print(f'https://astrogeology.usgs.gov/{link.get("src")}')

In [35]:
browser.visit("https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars")
soup_hemispheres = bs(browser.html, 'html.parser')
#Getting the titles of the hemispheres from h3 tag
hem_title = soup_hemispheres.find_all('h3')
for x in range(0, len(hem_title)):
    hem_title[x]= hem_title[x].text

dict={}
dict['title'] = []
dict['img_url'] = []
c=0
hem_url=[]

for titulo in hem_title:
    hemispheres={}
    browser.links.find_by_partial_text(titulo).click()  #hem_title[c].string).click() #this one worked
    #browser.links.find_by_partial_text(hem_title[c]).click() #this one worked
    #browser.find_by_name(hem_title[titulo].string).click() #this one did not work
    #hem_title[0].string
    html2 = browser.html
    soup_hd_img = bs(html2, 'html.parser')
    hd_img = soup_hd_img.find('a', target="_blank", text=('Sample')).get('href')   #['href']
    hemispheres['title']= titulo  #hem_title[c].string
    hemispheres['img_url']= hd_img
    hem_url.append(hemispheres)
    browser.back()
    c=c+1
    
hem_url

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [36]:
mars["hemispheres"]= hem_url

NameError: name 'mars' is not defined

In [ ]:
mars

In [ ]:
browser.quit()